<a href="https://colab.research.google.com/github/EvgenEgorov/transcriptomics_msu_HW/blob/main/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Семинар 1. Базовая работа с прочтениями

На этом семинаре мы разберём, как хранятся прочтения, какие есть открытые базы данных с прочтениями, а также то, как с ними правильно работать.

В данной части мы (для уменьшения времени выполнения) будем опираться на датасет <code>SRR3900953</code> (это не RNA-Seq!), однако абсолютно все те же процедуры можно выполнить и для RNA-Seq прочтений (про это будет домашнее задание).

## Загрузка прочтений

### SRA Toolkit

Для того, чтобы загрузить SRA-эксперимент и преобразовать его в .fastq-прочтения (мы сделаем это одновременно одной командой, однако можно выполнить их независимо), необходимо выполнить следующие команды. Сначала мы загрузим SRA Toolkit с сайта NCBI:

In [ ]:
!wget https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/3.0.0/sratoolkit.3.0.0-ubuntu64.tar.gz
!tar -xvzf sratoolkit.3.0.0-ubuntu64.tar.gz

Нам необходимо загрузить прочтения при помощи программы <code>fasterq-dump</code> из SRA Toolkit. Внутри Google Colab настроить SRA Toolkit правильно крайне сложно, поэтому необходимо будет выполнить некоторую странную процедуру: специально вызвать Segmentation fault у инструмента <code>vdb-config</code>.

In [ ]:
!sratoolkit.3.0.0-ubuntu64/bin/vdb-config -i

2022-11-16T12:18:13 vdb-config.3.0.0 fatal: SIGNAL - Segmentation fault 


Теперь мы можем спокойно запустить <code>fastq-dump</code> и разархивировать наши прочтения:

In [ ]:
!sratoolkit.3.0.0-ubuntu64/bin/fasterq-dump SRR3900953

spots read      : 4,941,237
reads read      : 9,882,474
reads written   : 9,882,474


### SRA Explorer

Однако кроме вариантов загрузки из SRA при помощи SRA Toolkit можно попробовать загрузить прочтения при помощи web-страницы <a href="https://sra-explorer.info/">SRA Explorer</a> &mdash; в таком случае пользователю просто будут указаны прямые ссылки на прочтения, а также много дополнительной информации. Однако не всегда это и правда поможет. Давайте посмотрим, какой результат будет при загрузке прочтений того же эксперимента из этого сервиса:

In [ ]:
!wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR390/003/SRR3900953/SRR3900953_1.fastq.gz
!wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR390/003/SRR3900953/SRR3900953_2.fastq.gz

Конкретно в данном случае всё хорошо, однако посмотрите на то, что предлагает загрузить SRA Explorer в случае с датасетом <code>SRR6410603</code> (из <a href="https://www.nature.com/articles/s41598-018-23226-4">статьи</a>, с которой вы будете работать дома). Иногда для того, чтобы конкретно распарсить SRA-файл, необходимо чётко обозначить, что файл с прочтениями нужно разбить на два &mdash; с прямыми и с обратными (в случае парноконцевого секвенирования). Здесь это сделано не было. Программа <code>fasterq-dump</code> это делает автоматически.

#### Задание

Прочитайте описание <a href="https://github.com/ncbi/sra-tools/wiki/HowTo:-fasterq-dump">функции fasterq-dump</a> и ответьте на вопрос: какой командой мы сможем получить вместо нескольких файлов с прочтениями (прямыми и обратными) всего один?

In [ ]:
fasterq-dump *sra acession* --concatenate-reads -include-technical

## Контроль качества прочтений

### FastQC

Для начала загрузим программу, при помощи которой мы будем проводить оценку качества, &mdash; <code>FastQC</code>.

In [ ]:
!wget https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.11.9.zip
!unzip fastqc_v0.11.9.zip

Затем сделаем скрипт <code>fastqc</code> выполняемым. В результате выполним анализ качества наших прочтений и положим результаты и все репорты в папку <code>qc</code>, которую перед этим создадим.

In [ ]:
!chmod +x FastQC/fastqc
!mkdir qc
!FastQC/fastqc -o qc -f fastq SRR3900953_1.fastq SRR3900953_2.fastq

**Задание**

Пожалуйста, посмотрите на .html-репорт, который выдаёт программа FastQC. Посмотрите, какие метрики контроля качества не проходят установленные в этой программе пороги. Объясните результаты. Посмотрите на результат работы FastQC на прочтениях RNA-Seq (загрузите при помощи команды `gdown --id 1tntpS8TBl3ERLMBYnso7tNiHeFVRmk8w`). В чём разница и почему?

In [ ]:
!gdown --id 1tntpS8TBl3ERLMBYnso7tNiHeFVRmk8w

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1tntpS8TBl3ERLMBYnso7tNiHeFVRmk8w
To: /content/RNA-Seq_FastQC_example.html
100% 668k/668k [00:00<00:00, 131MB/s]


**Ответ**

Порог качества в файлах SRR3900953_1.fastq и SRR3900953_2.fastq не проходят параметры Overrepresented sequences (перепредставленные последовательности - может быть как артефакт ПЦР, так и особенность копийности генов в нашем эксперименте) и Sequence Duplication Levels (дупликация последовательностей - причины могут быть те же самые, что и в предыдущем пункте). Также есть проблемы с соотношением нуклеотидов в ридах (вероятно, просто "случайные" праймеры отожглись совсем не случайно, поэтому в начале рида находятся неслучайные буквы).

По поводу разницы работы FastQC на прочтениях RNA-Seq - там, в отличие от наших предыдущих отчётов, хранится ещё информация о качестве ридов в зависимости от их положения на проточной ячейке, это важно для отслеживания качества оборудования, с помощью которого выполняют секвенирование

### MultiQC

Теперь установим программу MultiQC. Она необходима для того, чтобы комбинировать несколько различных отчётов в один. Запустим её, дав путь к директории с отчётами FastQC, и посмотрим на .html-отчёт, который будет получен в результате запуска.

In [ ]:
!pip3 install multiqc
!multiqc qc

В случае, когда вы работаете сразу с большим количеством образцов, это может значительно ускорить процесс анализа данных, а также легко понять, по какой причине те или иные точки могут быть аутлаерами (например, определить образцы с низким по сравнению с другими покрытием и проч.)

После этого можно загрузить .html-отчёт о качестве себе на компьютер и посмотреть, всё ли хорошо с прочтениями. Рассмотрим каждый пункт отчёта и возможные проблемы при анализе bulk RNA-Seq:
1. Sequence Quality Histograms
2. Per Sequence Quality Scores
3. <b>Per Base Sequence Content</b> &mdash; проблемы тут в случае RNA-Seq связаны с особенностью подготовки библиотеки (использовались случайные праймеры)
4. Per Sequence GC Content
5. Per Base N Content
6. Sequence Length Distribution
7. <b>Sequence Duplication Levels</b> &mdash; в основе порогов данной метрики лежит предположение, что высокое количество дублирующихся прочтений говорит об артефактах ПЦР (это справедливо для, например, геномных библиотек, в которых встретить два одинаковых рида очень маловероятно). Однако для RNA-Seq-экспериментов это нормально, потому что какие-то гены могут встречаться очень часто из-за высокой копийности их РНК (<a href="https://www.bioinformatics.babraham.ac.uk/projects/fastqc/Help/3%20Analysis%20Modules/8%20Duplicate%20Sequences.html" target="_blank">источник</a>)
8. <b>Overrepresented sequences</b> &mdash; проблема вызвана аналогичными седьмому пункту проблемами (<a href="https://www.bioinformatics.babraham.ac.uk/projects/fastqc/Help/3%20Analysis%20Modules/9%20Overrepresented%20Sequences.html" target="_blank">источник</a>)
9. <b>Adapter Content</b> &mdash; мы видим, что у нас в большом числе прочтений содержатся технические адаптеры. Однако почти все современные алгоритмы для работы с RNA-Seq (в т.ч. и kallisto и STAR) автоматически прорабатывают эту проблему, поэтому мануального устранения не требуется (<a href="https://dnatech.genomecenter.ucdavis.edu/faqs/when-should-i-trim-my-illumina-reads-and-how-should-i-do-it/" target="_blank">про то, когда и зачем лучше обрезать адаптеры</a>)

## Введение в Nexflow

NextFlow &mdash; это специальный язык, который позволяет вам запускать целые пайплайны из нескольких последовательно выполняемых программ, следя за тем, чтобы каждая стадия выполнялась строго одна за одной.

In [ ]:
!curl -fsSL https://get.nextflow.io | bash

Теперь создадим файл <code>my_pipeline.nf</code> со следующим содержанием:
```
params.SRA = "SRR000000"
params.results_dir = "results/"

log.info ""
log.info "  Q U A L I T Y   C O N T R O L  "
log.info "================================="
log.info "SRA number         : ${params.SRA}"
log.info "Results location   : ${params.results_dir}"
```

А теперь попробуйте передать различные значения флагов, которые вы описали выше, команде <code>nextflow run my_pipeline.nf</code>:

In [ ]:
!./nextflow run --SRA SRR3900953 my_pipeline.nf

N E X T F L O W  ~  version 22.10.2
Launching `my_pipeline.nf` [confident_pasteur] DSL2 - revision: a54422bfe5

  Q U A L I T Y   C O N T R O L  
SRA number         : SRR3900953
Results location   : results/


Теперь модифицируем наш файл так, чтобы он выполнял какой-то процесс. Например, загрузку прочтений по SRA-идентификатору:
```
params.SRA = "SRR000000"
params.results_dir = "results/"

log.info ""
log.info "  Q U A L I T Y   C O N T R O L  "
log.info "================================="
log.info "SRA number         : ${params.SRA}"
log.info "Results location   : ${params.results_dir}"

process DownloadFastQ {
  publishDir "${params.results_dir}"

  output:
    path "reads/*"

  script:
    """
    /content/sratoolkit.3.0.0-ubuntu64/bin/fasterq-dump ${params.SRA} -O reads/
    """
}

workflow {
  DownloadFastQ()
}
```

А теперь выполните скрипт и проследите за тем, какие файлы создаются в ходе выполнения скрипта:

In [ ]:
!./nextflow run my_pipeline.nf --SRA SRR3900953

N E X T F L O W  ~  version 22.10.2
Launching `my_pipeline.nf` [sad_venter] DSL2 - revision: 00b75134cc

  Q U A L I T Y   C O N T R O L  
SRA number         : SRR3900953
Results location   : results/
[-        ] process > DownloadFastQ -

executor >  local (1)
[52/f968a4] process > DownloadFastQ [  0%] 0 of 1

executor >  local (1)
[52/f968a4] process > DownloadFastQ [  0%] 0 of 1

executor >  local (1)
[52/f968a4] process > DownloadFastQ [100%] 1 of 1 ✔
Completed at: 16-Nov-2022 13:28:33
Duration    : 1m 16s
CPU hours   : (a few seconds)
Succeeded   : 1




Теперь модифицируем скрипт так, чтобы в итоговую директорию он сохранял как прочтения, так и результаты работы FastQC для всех прочтений:
```
params.SRA = "SRR000000"
params.results_dir = "results/"

log.info ""
log.info "  Q U A L I T Y   C O N T R O L  "
log.info "================================="
log.info "SRA number         : ${params.SRA}"
log.info "Results location   : ${params.results_dir}"

process DownloadFastQ {
  publishDir "${params.results_dir}"

  output:
    path "reads/*"

  script:
    """
    /content/sratoolkit.3.0.0-ubuntu64/bin/fasterq-dump ${params.SRA} -O reads/
    """
}

process QC {
  publishDir "${params.results_dir}"

  input:
    path x

  output:
    path "qc/*.html"

  script:
    """
    mkdir qc
    /content/FastQC/fastqc -o qc $x
    """
}

workflow {
  DownloadFastQ()
  QC( DownloadFastQ.out.collect() )
}
```

In [ ]:
!./nextflow run my_pipeline.nf --SRA SRR3900953

N E X T F L O W  ~  version 22.10.2
Launching `my_pipeline.nf` [naughty_goldwasser] DSL2 - revision: 4457d4ca78

  Q U A L I T Y   C O N T R O L  
SRA number         : SRR3900953
Results location   : results/
[-        ] process > DownloadFastQ -
[-        ] process > QC            -

[-        ] process > DownloadFastQ [  0%] 0 of 1
[-        ] process > QC            -

executor >  local (1)
[a2/7b01b2] process > DownloadFastQ [  0%] 0 of 1
[-        ] process > QC            -

executor >  local (2)
[a2/7b01b2] process > DownloadFastQ [100%] 1 of 1 ✔
[d1/025817] process > QC            [  0%] 0 of 1

executor >  local (2)
[a2/7b01b2] process > DownloadFastQ [100%] 1 of 1 ✔
[d1/025817] process > QC            [100%] 1 of 1 ✔
Completed at: 16-Nov-2022 13:32:21
Duration    : 1m 39s
CPU hours   : (a few seconds)
Succeeded   : 2




**Задание**

Добавьте в пайплайн выполнение MultiQC (чтобы в итоговой директории оказывался только отчёт MultiQC в .html, но без отчётов FastQC).

**Пайплайн**
```
params.results_dir = "results/"
SRA_list = params.SRA.split(",")

log.info ""
log.info "  Q U A L I T Y   C O N T R O L  "
log.info "================================="
log.info "SRA number         : ${SRA_list}"
log.info "Results location   : ${params.results_dir}"

process DownloadFastQ {
  publishDir "${params.results_dir}"

  input:
    val sra

  output:
    path "${sra}/*"

  script:
    """
    /content/sratoolkit.3.0.0-ubuntu64/bin/fasterq-dump ${sra} -O ${sra}/
    """
}

process QC {
  input:
    path x

  output:
    path "qc"

  script:
    """
    mkdir qc
    /content/FastQC/fastqc -o qc $x
    """
}

process MultiQC {
  publishDir "${params.results_dir}"

  input:
    path x

  output:
    path "multiqc_report.html"

  script:
    """
    multiqc $x
    """
}

workflow {
  data = Channel.of( SRA_list )
  DownloadFastQ(data)
  QC( DownloadFastQ.out )
  MultiQC( QC.out.collect() )
}
```

In [ ]:
!./nextflow run my_pipeline.nf --SRA SRR3900953

N E X T F L O W  ~  version 22.10.2
Launching `my_pipeline.nf` [drunk_heisenberg] DSL2 - revision: 8bb2b8fd7c

  Q U A L I T Y   C O N T R O L  
SRA number         : [SRR3900953]
Results location   : results/
[-        ] process > DownloadFastQ -
[-        ] process > QC            -
[-        ] process > MultiQC       -

executor >  local (1)
[18/ed8544] process > DownloadFastQ (1) [  0%] 0 of 1
[-        ] process > QC                -
[-        ] process > MultiQC           -

executor >  local (2)
[18/ed8544] process > DownloadFastQ (1) [100%] 1 of 1 ✔
[73/cacd4c] process > QC (1)            [  0%] 0 of 1
[-        ] process > MultiQC           -

executor >  local (2)
[18/ed8544] process > DownloadFastQ (1) [100%] 1 of 1 ✔
[73/cacd4c] process > QC (1)            [  0%] 0 of 1
[-        ] process > MultiQC           -

executor >  local (3)
[18/ed8544] process > DownloadFastQ (1) [100%] 1 of 1 ✔
[73/cacd4c] process > QC (1)            [100%] 1 of 1 ✔
[89/4ee1ee] process > MultiQC  

Для того, чтобы обкспечить параллельный запуск сразу нескольких процессов одним и тем же пайплайном, необходимо распарсить аргумент, который мы подаём в переменную <code>SRA</code> как список, после чего запустить наш немного модифицированный пайплайн:
```
params.results_dir = "results/"
SRA_list = params.SRA.split(",")

log.info ""
log.info "  Q U A L I T Y   C O N T R O L  "
log.info "================================="
log.info "SRA number         : ${SRA_list}"
log.info "Results location   : ${params.results_dir}"

process DownloadFastQ {
  publishDir "${params.results_dir}"

  input:
    val sra

  output:
    path "${sra}/*"

  script:
    """
    /content/sratoolkit.3.0.0-ubuntu64/bin/fasterq-dump ${sra} -O ${sra}/
    """
}

process QC {
  input:
    path x

  output:
    path "qc"

  script:
    """
    mkdir qc
    /content/FastQC/fastqc -o qc $x
    """
}

process MultiQC {
  publishDir "${params.results_dir}"

  input:
    path x

  output:
    path "multiqc_report.html"

  script:
    """
    multiqc $x
    """
}

workflow {
  data = Channel.of( SRA_list )
  DownloadFastQ(data)
  QC( DownloadFastQ.out )
  MultiQC( QC.out.collect() )
}
```
Для примера возьмём результаты секвенирования пост-ковидного кишечного метагенома (`SRR20851170`, `SRR20851171`, `SRR20851172`, `SRR20851173`, `SRR20851174`, `SRR20851176`, `SRR20851179`, `SRR20851183`).

In [ ]:
!./nextflow run my_pipeline.nf -with-report nextflow_report.html -with-dag flowchart.pdf --SRA SRR20851170,SRR20851171,SRR20851172,SRR20851173,SRR20851174,SRR20851176,SRR20851179,SRR20851183

N E X T F L O W  ~  version 22.10.2
Launching `my_pipeline.nf` [pedantic_woese] DSL2 - revision: 30656d3ce8

  Q U A L I T Y   C O N T R O L  
SRA number         : [SRR20851170, SRR20851171, SRR20851172, SRR20851173, SRR20851174, SRR20851176, SRR20851179, SRR20851183]
Results location   : results/
[-        ] process > DownloadFastQ -
[-        ] process > QC            -
[-        ] process > MultiQC       -

[-        ] process > DownloadFastQ [  0%] 0 of 8
[-        ] process > QC            -
[-        ] process > MultiQC       -

executor >  local (2)
[de/e8b2b5] process > DownloadFastQ (1) [  0%] 0 of 8
[-        ] process > QC                -
[-        ] process > MultiQC           -

executor >  local (3)
[8c/e6aef8] process > DownloadFastQ (3) [  0%] 0 of 8
[-        ] process > QC                -
[-        ] process > MultiQC           -

executor >  local (3)
[de/e8b2b5] process > DownloadFastQ (1) [ 12%] 1 of 8
[-        ] process > QC                [  0%] 0 of 1
[-     

**Задание**

Посмотрите на содержимое файов `flowchart.pdf` и `nextflow_report.html`. Что в них описано и как это может помочь при реализации пайплайна?

**Ответ на задание**

В файле `flowchart.pdf` представлена визуализация пайплайна в виде графа - может помочь для представления о порядке выполнения действий в нашем пайплайне, удобно.

В файле `nextflow_report.html` представлены технические характеристики каждого процесса  - подобный отчёт можно использовать, чтобы понять, какой процесс в пайплайне самый ресурсоёмкий, да и в принципе знать, как распределяется использование памяти и вычислительных ресурсов в пайплайне


## Выгрузка данных на GEO

Для того, чтобы выгрузить данные в общий доступ (а конкретно на GEO), необходимо заполнить таблицу, которую можно найти <a href="https://www.ncbi.nlm.nih.gov/geo/info/seq.html">тут</a>. После этого данные выгружаются на GEO и ставятся в очередь на проверку модераторами, которые одобряют или не одобряют ваш датасет.

#### Задание

Заполните таблицу (информацию, которую вы не знаете, можете просто придумать, но так, чтобы это было реалистично), которая бы описывала эксперимент и данные из эксперимента, с которым мы работаем в течение семинара.

Данные об эксперименте брал отсюда:<br>
'https://www.ncbi.nlm.nih.gov/sra/?term=SRR3900953'


**Study**: Population genetics of Oophaga sylvatica<br>
**Summary**: Here we investigate the phylogenetic and genomic differentiations among populations of Oophaga sylvatica, which exhibit one of the highest phenotypic diversification among poison frogs. We characterize 13 populations (12 monotypic and 1 polytypic) across the O. sylvatica distribution. These populations are mostly separated in two lineages distributed in the Northern and the Southern part of their range in Ecuador. We found relatively low genetic differentiation within each lineage, despite considerable phenotypic variation, and evidence for ongoing gene flow and genetic admixture among some populations of the Northern lineage. Overall these data suggests that phenotypical diversification and aposematic coloration novelty can arise in secondary contact zones even in systems where phenotypes are subject to strong stabilizing selection.<br>
**Contributors**: Harvard University<br>
**Experimental design**: ddRAD sequencing<br>


Детали опыта:

**Extract protocol**: DNA extraction kit (Zymogen) - это я уже тут сам придумываю.<br>
**Strategy**: OTHER<br>
**Source**: GENOMIC<br>
**Selection**: Restriction Digest<br>
**Layout**: PAIRED<br>

## Домашнее задание

Загрузите прочтения всех экспериментов из статьи <a href="https://www.nature.com/articles/s41598-018-23226-4"><b>Evaluation of two main RNA-seq approaches for gene quantification in clinical RNA sequencing: polyA+ selection versus rRNA depletion</b></a>, в которой сравнивают два протокола подготовки библиотек для bulk RNA-Seq: при помощи деплеции рРНК (=<code>total</code>) или при помощи селекции polyA-транскриптов (=<code>polyA+</code>).

При выполнении задания используйте Nextflow pipeline.

Так как все прочтения - это слишком большой объём данных, загрузим два эксперимента.
Скрипт для загрузки файлов:
```
params.results_dir = "results/"
SRA_list = params.SRA.split(",")

log.info ""
log.info "  D O W N L O A D I N G  "
log.info "================================="
log.info "SRA number         : ${SRA_list}"
log.info "Results location   : ${params.results_dir}"

process DownloadFastQ {
  publishDir "${params.results_dir}"

  input:
    val sra

  output:
    path "${sra}/*"

  script:
    """
    /content/sratoolkit.3.0.0-ubuntu64/bin/fasterq-dump ${sra} -O ${sra}/
    """
}

workflow {
  data = Channel.of( SRA_list )
  DownloadFastQ(data)
}


```

In [ ]:
!./nextflow run HW_pipeline.nf --SRA SRR6410603,	SRR6410604	

N E X T F L O W  ~  version 22.10.2
Launching `HW_pipeline.nf` [jovial_liskov] DSL2 - revision: d5614a9f4c

  D O W N L O A D I N G  
SRA number         : [SRR6410603]
Results location   : results/
[-        ] process > DownloadFastQ -

[-        ] process > DownloadFastQ [  0%] 0 of 1

executor >  local (1)
[31/a9525b] process > DownloadFastQ (1) [  0%] 0 of 1

executor >  local (1)
[31/a9525b] process > DownloadFastQ (1) [100%] 1 of 1 ✔

executor >  local (1)
[31/a9525b] process > DownloadFastQ (1) [100%] 1 of 1 ✔
Completed at: 16-Nov-2022 14:53:15
Duration    : 13m 30s
CPU hours   : 0.2
Succeeded   : 1


